In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from statsmodels.formula.api import ols


directory = "."
file_name= "After 1M, 3M FRA periods daily call compounding rate calculation.csv"

df = pd.read_csv(f'{directory}/{file_name}', index_col=0)
df['Date2'] = pd.to_datetime(df['Date2'])
df['annual_After_01M_3Mcall'] = df['After_01M_3Mcall']*4
df

,Date2,call_close,01M*04M_MID,After_01M_3Mcall,annual_After_01M_3Mcall
0,2022-12-30,3.212,3.970,0.859263,3.437053
1,2022-12-29,3.212,3.970,0.859197,3.436788
2,2022-12-28,3.187,3.910,0.859131,3.436523
3,2022-12-27,3.252,4.010,0.859064,3.436257
4,2022-12-26,3.269,4.050,0.858888,3.435550
...,...,...,...,...,...
2864,2012-01-06,3.250,3.530,0.813087,3.252349
2865,2012-01-05,3.250,3.530,0.813087,3.252349
2866,2012-01-04,3.240,3.505,0.813087,3.252349
2867,2012-01-03,3.250,3.530,0.813087,3.252349


In [182]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2869 entries, 0 to 2868
Data columns (total 7 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   Date2                    2869 non-null   datetime64[ns]
 1   call_close               2869 non-null   float64       
 2   01M*04M_MID              2869 non-null   float64       
 3   After_01M_3Mcall         2869 non-null   float64       
 4   annual_After_01M_3Mcall  2869 non-null   float64       
 5   X_variable               2869 non-null   float64       
 6   Y_variable               2869 non-null   float64       
dtypes: datetime64[ns](1), float64(6)
memory usage: 243.9 KB


In [3]:
for idx in df.index:

    # final X, Y variables for linear regresssion 
    # X is FRA - daily call close rate
    #Y is daily compounding call rate - daily call close rate
    df.loc[idx, 'X_variable'] = df.loc[idx, '01M*04M_MID'] - df.loc[idx, 'call_close']
    df.loc[idx, 'Y_variable'] = df.loc[idx, 'annual_After_01M_3Mcall'] - df.loc[idx, 'call_close']
    
df

,Date2,call_close,01M*04M_MID,After_01M_3Mcall,annual_After_01M_3Mcall,X_variable,Y_variable
0,2022-12-30,3.212,3.970,0.859263,3.437053,0.758,0.225053
1,2022-12-29,3.212,3.970,0.859197,3.436788,0.758,0.224788
2,2022-12-28,3.187,3.910,0.859131,3.436523,0.723,0.249523
3,2022-12-27,3.252,4.010,0.859064,3.436257,0.758,0.184257
4,2022-12-26,3.269,4.050,0.858888,3.435550,0.781,0.166550
...,...,...,...,...,...,...,...
2864,2012-01-06,3.250,3.530,0.813087,3.252349,0.280,0.002349
2865,2012-01-05,3.250,3.530,0.813087,3.252349,0.280,0.002349
2866,2012-01-04,3.240,3.505,0.813087,3.252349,0.265,0.012349
2867,2012-01-03,3.250,3.530,0.813087,3.252349,0.280,0.002349


In [4]:
# regresssion

df2 = df[['Y_variable', 'X_variable']]
res = ols(formula = 'Y_variable ~ X_variable', data = df2).fit()
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             Y_variable   R-squared:                       0.457
Model:                            OLS   Adj. R-squared:                  0.457
Method:                 Least Squares   F-statistic:                     2415.
Date:                Sun, 21 May 2023   Prob (F-statistic):               0.00
Time:                        15:47:45   Log-Likelihood:                 1242.2
No. Observations:                2869   AIC:                            -2480.
Df Residuals:                    2867   BIC:                            -2468.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.1632      0.004    -36.567      0.000      -0.172      -0.154
X_variable     0.6879      0.014     49.138      0.000       0.660       0.715
==============================================================================
Omnibus:                      154.431   Durbin-Watson:                   0.348
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              598.166
Skew:                           0.039   Prob(JB):                    1.29e-130
Kurtosis:                       5.236   Cond. No.                         5.06
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [5]:
'''
일단 연율인지 전부 다 맞추고
즉 콜누적 일복리 금리를 연복리로 전환시켜주고
플롯 찍어보고,
alpha는 이미 들어있고,
ARIMA 관점에서 생각해보고
'''

'\n일단 연율인지 전부 다 맞추고\n즉 콜누적 일복리 금리를 연복리로 전환시켜주고\n플롯 찍어보고,\nalpha는 이미 들어있고,\nARIMA 관점에서 생각해보고\n'

- Weekend는 제외되어 있는 상태?
- holiday 적용
- 그 사이 기간 일복리 계산 how?

---

# 비영업일 classifier
- module
    - QuantLib
    - pytimekr


- type
    - datetime
    - pandas timestamp
    - ql.Date

## QuantLib

In [12]:
import QuantLib as ql

In [40]:
kr = ql.SouthKorea(0)

In [340]:
t0 = df.iloc[-1]['Date2']
t1 = df.iloc[0]['Date2']

date1, date2

date0 = ql.Date(t0.day, t0.month, t0.year)
date1 = ql.Date(t1.day, t1.month, t1.year)

kr_nonBusiness = kr.holidayList(date0, date1)
print(kr_nonBusiness)

(Date(23,1,2012), Date(24,1,2012), Date(1,3,2012), Date(11,4,2012), Date(1,5,2012), Date(28,5,2012), Date(6,6,2012), Date(15,8,2012), Date(1,10,2012), Date(3,10,2012), Date(19,12,2012), Date(25,12,2012), Date(1,1,2013), Date(11,2,2013), Date(1,3,2013), Date(1,5,2013), Date(17,5,2013), Date(6,6,2013), Date(15,8,2013), Date(18,9,2013), Date(19,9,2013), Date(20,9,2013), Date(3,10,2013), Date(9,10,2013), Date(25,12,2013), Date(1,1,2014), Date(30,1,2014), Date(31,1,2014), Date(1,5,2014), Date(5,5,2014), Date(6,5,2014), Date(4,6,2014), Date(6,6,2014), Date(15,8,2014), Date(8,9,2014), Date(9,9,2014), Date(10,9,2014), Date(3,10,2014), Date(9,10,2014), Date(25,12,2014), Date(1,1,2015), Date(18,2,2015), Date(19,2,2015), Date(20,2,2015), Date(1,5,2015), Date(5,5,2015), Date(25,5,2015), Date(14,8,2015), Date(28,9,2015), Date(29,9,2015), Date(9,10,2015), Date(25,12,2015), Date(1,1,2016), Date(8,2,2016), Date(9,2,2016), Date(10,2,2016), Date(1,3,2016), Date(13,4,2016), Date(5,5,2016), Date(6,6,2016)

- 완벽하지 않은 부분을 manual하게 리스트에 add, remove 해줄 수 있다.
    - 주말은 추가 안됨, weekday만 추가 가능

In [397]:
kr_holidayList = kr.holidayList(ql.Date(1,1,2023), ql.Date(31,12,2023))
kr.addHoliday(ql.Date(1, 3, 2023))
kr.removeHoliday(ql.Date(3, 2, 2023))
kr.holidayList(ql.Date(1,1,2023), ql.Date(31,12,2023))

(Date(23,1,2023),
 Date(1,3,2023),
 Date(1,5,2023),
 Date(5,5,2023),
 Date(26,5,2023),
 Date(6,6,2023),
 Date(15,8,2023),
 Date(28,9,2023),
 Date(29,9,2023),
 Date(3,10,2023),
 Date(9,10,2023),
 Date(25,12,2023))

## pytimekr

https://developertools.tistory.com/entry/PYHTON-%ED%8C%8C%EC%9D%B4%EC%8D%AC-%EA%B3%B5%ED%9C%B4%EC%9D%BC-%ED%99%95%EC%9D%B8-%EB%B0%8F-%EC%98%A4%EB%8A%98%EC%9D%B4-%ED%8F%89%EC%9D%BC%EC%9D%B8%EC%A7%80-%EC%97%AC%EB%B6%80-%ED%99%95%EC%9D%B8-pytimekr-datetime

https://pypi.org/project/pytimekr/

In [398]:
# !pip install pytimekr
from pytimekr import pytimekr

In [338]:
pytimekr.is_red_day(datetime(2023,5,21))

True

In [69]:
pytimekr.is_red_day(datetime(2023,5,22))

False

In [122]:
list = pytimekr.holidays() #holidays메소드는 리스트 형태로 관련값 반환

for i in list:
    print(i)

2023-01-21
2023-01-22
2023-01-23
2023-09-28
2023-09-29
2023-09-30
2023-01-01
2023-03-01
2023-05-05
2023-05-26
2023-06-06
2023-08-15
2023-10-09
2023-10-03
2023-12-25


In [111]:
pytimekr.is_red_day(datetime(2022,8,15))

False

In [130]:
# pytimekr.holidays()의 argument로 year 넣으면 해당 연도의 holidays 일괄 출력
pytimekr.holidays(2000) 

[datetime.date(2000, 2, 4),
 datetime.date(2000, 2, 5),
 datetime.date(2000, 2, 6),
 datetime.date(2000, 9, 11),
 datetime.date(2000, 9, 12),
 datetime.date(2000, 9, 13),
 datetime.date(2000, 1, 1),
 datetime.date(2000, 3, 1),
 datetime.date(2000, 5, 5),
 datetime.date(2000, 5, 11),
 datetime.date(2000, 6, 6),
 datetime.date(2000, 8, 15),
 datetime.date(2000, 10, 9),
 datetime.date(2000, 10, 3),
 datetime.date(2000, 12, 25)]

### QuantLib, pytimekr 비교

In [113]:
list = pytimekr.holidays() 
list

# pytimekr은 공휴일과 주말 겹쳐도 출력

[datetime.date(2023, 1, 21),
 datetime.date(2023, 1, 22),
 datetime.date(2023, 1, 23),
 datetime.date(2023, 9, 28),
 datetime.date(2023, 9, 29),
 datetime.date(2023, 9, 30),
 datetime.date(2023, 1, 1),
 datetime.date(2023, 3, 1),
 datetime.date(2023, 5, 5),
 datetime.date(2023, 5, 26),
 datetime.date(2023, 6, 6),
 datetime.date(2023, 8, 15),
 datetime.date(2023, 10, 9),
 datetime.date(2023, 10, 3),
 datetime.date(2023, 12, 25)]

In [114]:
t0 = datetime(2023,1,1)
t1 = datetime(2023,12,31)

date1, date2

date0 = ql.Date(t0.day, t0.month, t0.year)
date1 = ql.Date(t1.day, t1.month, t1.year)

kr_nonBusiness = kr.holidayList(date0, date1)
print(kr_nonBusiness)

# quantlib은 공휴일이라도 주말은 제외

(Date(23,1,2023), Date(1,3,2023), Date(1,5,2023), Date(5,5,2023), Date(26,5,2023), Date(6,6,2023), Date(15,8,2023), Date(28,9,2023), Date(29,9,2023), Date(3,10,2023), Date(9,10,2023), Date(25,12,2023))


2023.1.24 대체공휴일은 반영안됨

In [120]:
len(list), len(kr_nonBusiness)

(15, 12)

In [118]:
for i in list:
    for j in kr_nonBusiness:
        if i == j.to_date():
            print(i, j)

2023-01-23 January 23rd, 2023
2023-09-28 September 28th, 2023
2023-09-29 September 29th, 2023
2023-03-01 March 1st, 2023
2023-05-05 May 5th, 2023
2023-05-26 May 26th, 2023
2023-06-06 June 6th, 2023
2023-08-15 August 15th, 2023
2023-10-09 October 9th, 2023
2023-10-03 October 3rd, 2023
2023-12-25 December 25th, 2023


In [58]:
list

[datetime.date(2023, 1, 21),
 datetime.date(2023, 1, 22),
 datetime.date(2023, 1, 23),
 datetime.date(2023, 9, 28),
 datetime.date(2023, 9, 29),
 datetime.date(2023, 9, 30),
 datetime.date(2023, 1, 1),
 datetime.date(2023, 3, 1),
 datetime.date(2023, 5, 5),
 datetime.date(2023, 5, 26),
 datetime.date(2023, 6, 6),
 datetime.date(2023, 8, 15),
 datetime.date(2023, 10, 9),
 datetime.date(2023, 10, 3),
 datetime.date(2023, 12, 25)]

https://appia.tistory.com/760

In [59]:
pytimekr.christmas()

datetime.date(2023, 12, 25)

### QuantLib과 pytimekr에서 추출한 공휴일 합치기

1. 2023년 기준

In [132]:
pytimekr_holidays = pytimekr.holidays() 
pytimekr_holidays

[datetime.date(2023, 1, 21),
 datetime.date(2023, 1, 22),
 datetime.date(2023, 1, 23),
 datetime.date(2023, 9, 28),
 datetime.date(2023, 9, 29),
 datetime.date(2023, 9, 30),
 datetime.date(2023, 1, 1),
 datetime.date(2023, 3, 1),
 datetime.date(2023, 5, 5),
 datetime.date(2023, 5, 26),
 datetime.date(2023, 6, 6),
 datetime.date(2023, 8, 15),
 datetime.date(2023, 10, 9),
 datetime.date(2023, 10, 3),
 datetime.date(2023, 12, 25)]

In [135]:
t0 = datetime(2023,1,1)
t1 = datetime(2023,12,31)
date0 = ql.Date(t0.day, t0.month, t0.year)
date1 = ql.Date(t1.day, t1.month, t1.year)

quantlib_holidays = kr.holidayList(date0, date1)
print(quantlib_holidays)

# quantlib은 공휴일이라도 주말은 제외

quantlib_holidays_list = [i.to_date() for i in quantlib_holidays]
quantlib_holidays_list

(Date(23,1,2023), Date(1,3,2023), Date(1,5,2023), Date(5,5,2023), Date(26,5,2023), Date(6,6,2023), Date(15,8,2023), Date(28,9,2023), Date(29,9,2023), Date(3,10,2023), Date(9,10,2023), Date(25,12,2023))


[datetime.date(2023, 1, 23),
 datetime.date(2023, 3, 1),
 datetime.date(2023, 5, 1),
 datetime.date(2023, 5, 5),
 datetime.date(2023, 5, 26),
 datetime.date(2023, 6, 6),
 datetime.date(2023, 8, 15),
 datetime.date(2023, 9, 28),
 datetime.date(2023, 9, 29),
 datetime.date(2023, 10, 3),
 datetime.date(2023, 10, 9),
 datetime.date(2023, 12, 25)]

In [143]:
len(pytimekr_holidays), len(quantlib_holidays_list)

(15, 12)

In [165]:
holidays_list = pytimekr_holidays +quantlib_holidays_list
holidays_list = [*set(holidays)]     # 단순 concat된 list를 set에 넣어서 중복제거
len(holidays_list)

16

In [166]:
holidays_list

[datetime.date(2023, 1, 23),
 datetime.date(2023, 5, 5),
 datetime.date(2023, 1, 22),
 datetime.date(2023, 8, 15),
 datetime.date(2023, 9, 28),
 datetime.date(2023, 6, 6),
 datetime.date(2023, 12, 25),
 datetime.date(2023, 1, 21),
 datetime.date(2023, 1, 1),
 datetime.date(2023, 5, 1),
 datetime.date(2023, 10, 9),
 datetime.date(2023, 3, 1),
 datetime.date(2023, 9, 29),
 datetime.date(2023, 10, 3),
 datetime.date(2023, 9, 30),
 datetime.date(2023, 5, 26)]

In [228]:
t0 = datetime(2021,1,1)
t1 = datetime(2022,12,31)
date0 = ql.Date(t0.day, t0.month, t0.year)
date1 = ql.Date(t1.day, t1.month, t1.year)

quantlib_holidays = kr.holidayList(date0, date1)
print(quantlib_holidays)

(Date(1,1,2021), Date(11,2,2021), Date(12,2,2021), Date(1,3,2021), Date(5,5,2021), Date(19,5,2021), Date(20,9,2021), Date(21,9,2021), Date(22,9,2021), Date(31,1,2022), Date(1,2,2022), Date(2,2,2022), Date(1,3,2022), Date(5,5,2022), Date(6,6,2022), Date(15,8,2022), Date(9,9,2022), Date(3,10,2022))


In [229]:
[i.to_date() for i in quantlib_holidays]

[datetime.date(2021, 1, 1),
 datetime.date(2021, 2, 11),
 datetime.date(2021, 2, 12),
 datetime.date(2021, 3, 1),
 datetime.date(2021, 5, 5),
 datetime.date(2021, 5, 19),
 datetime.date(2021, 9, 20),
 datetime.date(2021, 9, 21),
 datetime.date(2021, 9, 22),
 datetime.date(2022, 1, 31),
 datetime.date(2022, 2, 1),
 datetime.date(2022, 2, 2),
 datetime.date(2022, 3, 1),
 datetime.date(2022, 5, 5),
 datetime.date(2022, 6, 6),
 datetime.date(2022, 8, 15),
 datetime.date(2022, 9, 9),
 datetime.date(2022, 10, 3)]

In [246]:
for y in range(2011,2022):
    for j in pytimekr.holidays(y):
        print(j)
    

2011-02-02
2011-02-03
2011-02-04
2011-09-11
2011-09-12
2011-09-13
2011-01-01
2011-03-01
2011-05-05
2011-05-10
2011-06-06
2011-08-15
2011-10-09
2011-10-03
2011-12-25
2012-01-22
2012-01-23
2012-01-24
2012-09-29
2012-09-30
2012-10-01
2012-01-01
2012-03-01
2012-05-05
2012-04-28
2012-06-06
2012-08-15
2012-10-09
2012-10-03
2012-12-25
2013-02-09
2013-02-10
2013-02-11
2013-09-18
2013-09-19
2013-09-20
2013-01-01
2013-03-01
2013-05-05
2013-05-17
2013-06-06
2013-08-15
2013-10-09
2013-10-03
2013-12-25
2014-01-30
2014-01-31
2014-02-01
2014-09-07
2014-09-08
2014-09-09
2014-01-01
2014-03-01
2014-05-05
2014-05-06
2014-06-06
2014-08-15
2014-10-09
2014-10-03
2014-12-25
2015-02-18
2015-02-19
2015-02-20
2015-09-26
2015-09-27
2015-09-28
2015-01-01
2015-03-01
2015-05-05
2015-05-25
2015-06-06
2015-08-15
2015-10-09
2015-10-03
2015-12-25
2016-02-07
2016-02-08
2016-02-09
2016-09-14
2016-09-15
2016-09-16
2016-01-01
2016-03-01
2016-05-05
2016-05-14
2016-06-06
2016-08-15
2016-10-09
2016-10-03
2016-12-25
2017-01-27

In [243]:
[for j in pytimekr.holidays(y) for y in range(2011, 2022)]

NameError: name 'y' is not defined

함수화

In [260]:
def gen_holidayList(y0, y1):
    
    pytimekr_holidaysList = []
    
    for i in range(y0,y1):
        for j in pytimekr.holidays(i):
            pytimekr_holidaysList.append(j)
    
    

    t0 = datetime(y0,1,1); date0 = ql.Date(t0.day, t0.month, t0.year)
    t1 = datetime(y1,12,31); date1 = ql.Date(t1.day, t1.month, t1.year)
    quantlib_holidays = kr.holidayList(date0, date1)
    quantlib_holidaysList = [i.to_date() for i in quantlib_holidays]
    
    
    holidays_list = pytimekr_holidaysList + quantlib_holidaysList
    holidays_list = [*set(holidays_list)]     # 단순 concat된 list를 set에 넣어서 중복제거
    
    print(f'pydatekr: {len(pytimekr_holidaysList)}, QuantLib: {len(quantlib_holidaysList)}, Total: {len(holidays_list)}')
    
    return  holidays_list

In [320]:
gen_holidayList(2012,2022)

pydatekr: 150, QuantLib: 141, Total: 185


[datetime.date(2012, 1, 23),
 datetime.date(2021, 12, 25),
 datetime.date(2015, 12, 25),
 datetime.date(2020, 1, 1),
 datetime.date(2012, 12, 19),
 datetime.date(2014, 5, 5),
 datetime.date(2018, 6, 6),
 datetime.date(2020, 9, 30),
 datetime.date(2018, 2, 15),
 datetime.date(2021, 9, 21),
 datetime.date(2016, 2, 9),
 datetime.date(2012, 3, 1),
 datetime.date(2013, 2, 10),
 datetime.date(2018, 6, 13),
 datetime.date(2022, 3, 1),
 datetime.date(2019, 3, 1),
 datetime.date(2017, 1, 29),
 datetime.date(2020, 12, 25),
 datetime.date(2018, 5, 1),
 datetime.date(2015, 9, 27),
 datetime.date(2016, 3, 1),
 datetime.date(2012, 4, 28),
 datetime.date(2017, 6, 6),
 datetime.date(2018, 8, 15),
 datetime.date(2020, 1, 24),
 datetime.date(2015, 2, 18),
 datetime.date(2017, 5, 9),
 datetime.date(2015, 9, 28),
 datetime.date(2020, 8, 15),
 datetime.date(2012, 4, 11),
 datetime.date(2020, 3, 1),
 datetime.date(2018, 5, 7),
 datetime.date(2022, 2, 2),
 datetime.date(2012, 8, 15),
 datetime.date(2018, 3, 

In [324]:
for i in holidays_list:
    print(i.strftime("%y-%m-%d"))

12-01-23
21-12-25
15-12-25
20-01-01
12-12-19
14-05-05
18-06-06
20-09-30
18-02-15
21-09-21
16-02-09
12-03-01
13-02-10
18-06-13
22-03-01
19-03-01
17-01-29
20-12-25
18-05-01
15-09-27
16-03-01
12-04-28
17-06-06
18-08-15
20-01-24
15-02-18
17-05-09
15-09-28
20-08-15
12-04-11
20-03-01
18-05-07
22-02-02
12-08-15
18-03-01
13-09-20
20-06-06
12-05-01
18-09-23
16-02-10
19-02-06
20-05-05
17-05-05
19-02-04
18-05-22
13-02-11
15-03-01
21-09-20
14-01-30
19-12-25
18-09-24
12-10-03
12-09-29
16-08-15
21-09-22
15-05-25
14-09-07
16-05-05
19-10-09
20-10-01
21-02-11
20-01-26
14-10-09
14-09-08
18-10-09
17-10-03
13-05-17
19-05-12
15-02-19
14-02-01
13-08-15
21-05-19
17-05-01
12-10-01
14-05-06
18-05-05
16-04-13
19-01-01
14-08-15
15-01-01
13-05-05
17-08-15
12-01-24
18-09-25
12-09-30
13-02-09
14-01-01
16-09-15
15-08-15
18-02-17
17-10-06
18-10-03
15-09-29
12-05-28
14-06-04
16-10-09
20-04-15
18-01-01
19-09-13
17-01-28
14-01-31
17-01-30
18-09-26
15-10-09
16-02-08
20-05-01
12-01-01
20-01-25
16-05-14
14-05-01
16-06-06
1

### 비영업일 Classifier

In [335]:
def is_holiday(date, holidays_list):
    
    day = date.weekday()
    
    for i in holidays_list:
            if(date.strftime("%y-%m-%d") == i.strftime("%y-%m-%d")): 
                return True
    
    # 먼저 주말 여부 체크
    if day > 4:
        return True   
    else:
        return False

In [336]:
holidays_list = gen_holidayList(2012, 2022)

pydatekr: 150, QuantLib: 141, Total: 185


현재 data에 적용

In [351]:
for i in df['Date2']:
    print(i.strftime("%y-%m-%d"), is_holiday(i, holidays_list))

22-12-30 False
22-12-29 False
22-12-28 False
22-12-27 False
22-12-26 False
22-12-23 False
22-12-22 False
22-12-21 False
22-12-20 False
22-12-19 False
22-12-16 False
22-12-15 False
22-12-14 False
22-12-13 False
22-12-12 False
22-12-09 False
22-12-08 False
22-12-07 False
22-12-06 False
22-12-05 False
22-12-02 False
22-12-01 False
22-11-30 False
22-11-29 False
22-11-28 False
22-11-25 False
22-11-24 False
22-11-23 False
22-11-22 False
22-11-21 False
22-11-18 False
22-11-17 False
22-11-16 False
22-11-15 False
22-11-14 False
22-11-11 False
22-11-10 False
22-11-09 False
22-11-08 False
22-11-07 False
22-11-04 False
22-11-03 False
22-11-02 False
22-11-01 False
22-10-31 False
22-10-28 False
22-10-27 False
22-10-26 False
22-10-25 False
22-10-24 False
22-10-21 False
22-10-20 False
22-10-19 False
22-10-18 False
22-10-17 False
22-10-14 False
22-10-13 False
22-10-12 False
22-10-11 False
22-10-10 False
22-10-07 False
22-10-06 False
22-10-05 False
22-10-04 False
22-10-03 True
22-09-30 False
22-09-29 Fa

모든날짜에 적용

In [337]:
start_date = pd.to_datetime('2012-01-01') ## 시작 날짜
end_date = pd.to_datetime('2022-12-31') ## 마지막 날짜
 
dates = pd.date_range(start_date,end_date,freq='D') ## 일단위로 생성


for i in dates:
    print(i.strftime("%y-%m-%d"), is_holiday(i, holidays_list))

12-01-01 True
12-01-02 False
12-01-03 False
12-01-04 False
12-01-05 False
12-01-06 False
12-01-07 True
12-01-08 True
12-01-09 False
12-01-10 False
12-01-11 False
12-01-12 False
12-01-13 False
12-01-14 True
12-01-15 True
12-01-16 False
12-01-17 False
12-01-18 False
12-01-19 False
12-01-20 False
12-01-21 True
12-01-22 True
12-01-23 True
12-01-24 True
12-01-25 False
12-01-26 False
12-01-27 False
12-01-28 True
12-01-29 True
12-01-30 False
12-01-31 False
12-02-01 False
12-02-02 False
12-02-03 False
12-02-04 True
12-02-05 True
12-02-06 False
12-02-07 False
12-02-08 False
12-02-09 False
12-02-10 False
12-02-11 True
12-02-12 True
12-02-13 False
12-02-14 False
12-02-15 False
12-02-16 False
12-02-17 False
12-02-18 True
12-02-19 True
12-02-20 False
12-02-21 False
12-02-22 False
12-02-23 False
12-02-24 False
12-02-25 True
12-02-26 True
12-02-27 False
12-02-28 False
12-02-29 False
12-03-01 True
12-03-02 False
12-03-03 True
12-03-04 True
12-03-05 False
12-03-06 False
12-03-07 False
12-03-08 False
12

대체공휴일은 잘 잡아내지 못함 -> list에 추가하는 방식으로 cover

In [339]:
for i in dates:
    print(i, pytimekr.is_red_day(i)) # is_red_day는 주말 파악하는 method로 추론

2012-01-01 00:00:00 True
2012-01-02 00:00:00 False
2012-01-03 00:00:00 False
2012-01-04 00:00:00 False
2012-01-05 00:00:00 False
2012-01-06 00:00:00 False
2012-01-07 00:00:00 True
2012-01-08 00:00:00 True
2012-01-09 00:00:00 False
2012-01-10 00:00:00 False
2012-01-11 00:00:00 False
2012-01-12 00:00:00 False
2012-01-13 00:00:00 False
2012-01-14 00:00:00 True
2012-01-15 00:00:00 True
2012-01-16 00:00:00 False
2012-01-17 00:00:00 False
2012-01-18 00:00:00 False
2012-01-19 00:00:00 False
2012-01-20 00:00:00 False
2012-01-21 00:00:00 True
2012-01-22 00:00:00 True
2012-01-23 00:00:00 False
2012-01-24 00:00:00 False
2012-01-25 00:00:00 False
2012-01-26 00:00:00 False
2012-01-27 00:00:00 False
2012-01-28 00:00:00 True
2012-01-29 00:00:00 True
2012-01-30 00:00:00 False
2012-01-31 00:00:00 False
2012-02-01 00:00:00 False
2012-02-02 00:00:00 False
2012-02-03 00:00:00 False
2012-02-04 00:00:00 True
2012-02-05 00:00:00 True
2012-02-06 00:00:00 False
2012-02-07 00:00:00 False
2012-02-08 00:00:00 Fal

C:\Users\jjgus\anaconda3\lib\site-packages\pytimekr\pytimekr.py:439: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  if date == holiday:


 False
2014-08-22 00:00:00 False
2014-08-23 00:00:00 True
2014-08-24 00:00:00 True
2014-08-25 00:00:00 False
2014-08-26 00:00:00 False
2014-08-27 00:00:00 False
2014-08-28 00:00:00 False
2014-08-29 00:00:00 False
2014-08-30 00:00:00 True
2014-08-31 00:00:00 True
2014-09-01 00:00:00 False
2014-09-02 00:00:00 False
2014-09-03 00:00:00 False
2014-09-04 00:00:00 False
2014-09-05 00:00:00 False
2014-09-06 00:00:00 True
2014-09-07 00:00:00 True
2014-09-08 00:00:00 False
2014-09-09 00:00:00 False
2014-09-10 00:00:00 False
2014-09-11 00:00:00 False
2014-09-12 00:00:00 False
2014-09-13 00:00:00 True
2014-09-14 00:00:00 True
2014-09-15 00:00:00 False
2014-09-16 00:00:00 False
2014-09-17 00:00:00 False
2014-09-18 00:00:00 False
2014-09-19 00:00:00 False
2014-09-20 00:00:00 True
2014-09-21 00:00:00 True
2014-09-22 00:00:00 False
2014-09-23 00:00:00 False
2014-09-24 00:00:00 False
2014-09-25 00:00:00 False
2014-09-26 00:00:00 False
2014-09-27 00:00:00 True
2014-09-28 00:00:00 True
2014-09-29 00:00:

df 데이터에 적용

---

# Quantlib datetime timestamp비교
- strftime으로 str 타입으로 형식 통일하는 방식으로 같은지 여부 파악

In [313]:
# Quantlib
ql_dt = ql.Date(22,12,2019)
ql_dt = str(ql_dt.to_date().strftime("%y-%m-%d"))

In [314]:
# pandas timestamp
pd_dt = pd.Timestamp(2019, 12, 22).strftime("%y-%m-%d")
type(pt_2), pt_dt

(str, '19-12-22')

In [315]:
# datetime
dt_dt = datetime(2019,12,22).strftime("%y-%m-%d")
type(dt_dt), dt_dt

(str, '19-12-22')

In [318]:
print(ql_dt == pt_dt)
print(ql_dt == dt_dt)
print(pt_dt == dt_dt)

True
True
True


# 기존 data에 공휴일 remove

In [352]:
df['Date2']

0      2022-12-30
1      2022-12-29
2      2022-12-28
3      2022-12-27
4      2022-12-26
          ...    
2864   2012-01-06
2865   2012-01-05
2866   2012-01-04
2867   2012-01-03
2868   2012-01-02
Name: Date2, Length: 2869, dtype: datetime64[ns]

In [366]:
len(df)

2869

In [368]:
for i, row in df.iterrows():
    date_str = row['Date2'].strftime("%y-%m-%d")
    if is_holiday(row['Date2'], holidays_list) == True:
        print(i, date_str, is_holiday(row['Date2'], holidays_list))
        df.drop(i, axis=0, inplace=True)

64 22-10-03 True
80 22-09-09 True
99 22-08-15 True
149 22-06-06 True
171 22-05-05 True
218 22-03-01 True
237 22-02-02 True
238 22-02-01 True
239 22-01-31 True
332 21-09-22 True
333 21-09-21 True
334 21-09-20 True
422 21-05-19 True
432 21-05-05 True
479 21-03-01 True
490 21-02-12 True
491 21-02-11 True
520 21-01-01 True
525 20-12-25 True
580 20-10-09 True
585 20-10-02 True
586 20-10-01 True
587 20-09-30 True
619 20-08-17 True
693 20-05-05 True
695 20-05-01 True
696 20-04-30 True
707 20-04-15 True
764 20-01-27 True
765 20-01-24 True
782 20-01-01 True
787 19-12-25 True
842 19-10-09 True
846 19-10-03 True
860 19-09-13 True
861 19-09-12 True
881 19-08-15 True
931 19-06-06 True
954 19-05-06 True
957 19-05-01 True
1000 19-03-01 True
1017 19-02-06 True
1018 19-02-05 True
1019 19-02-04 True
1043 19-01-01 True
1048 18-12-25 True
1103 18-10-09 True
1107 18-10-03 True
1112 18-09-26 True
1113 18-09-25 True
1114 18-09-24 True
1142 18-08-15 True
1187 18-06-13 True
1192 18-06-06 True
1203 18-05-22 Tru

In [369]:
len(df)

2727

row 개수 2869 -> 2727개

In [372]:
for i in df.index:
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285